## DATASET BANK LOAN 

In [212]:
import pandas as pd
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_selection import VarianceThreshold, SelectKBest, mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import GenericUnivariateSelect
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.base import clone
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

data=pd.read_csv("Bank_Personal_Loan_Modelling.csv",sep=',')
data

,ID,Age,Experience,Income,ZIP Code,Family,CCAvg,Education,Mortgage,Personal Loan,Securities Account,CD Account,Online,CreditCard
0,1,25,1,49,91107,4,1.6,1,0,0,1,0,0,0
1,2,45,19,34,90089,3,1.5,1,0,0,1,0,0,0
2,3,39,15,11,94720,1,1.0,1,0,0,0,0,0,0
3,4,35,9,100,94112,1,2.7,2,0,0,0,0,0,0
4,5,35,8,45,91330,4,1.0,2,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4996,29,3,40,92697,1,1.9,3,0,0,0,0,1,0
4996,4997,30,4,15,92037,4,0.4,1,85,0,0,0,1,0
4997,4998,63,39,24,93023,2,0.3,3,0,0,0,0,0,0
4998,4999,65,40,49,90034,3,0.5,2,0,0,0,0,1,0


In [112]:
data.isnull().sum()

ID                    0
Age                   0
Experience            0
Income                0
ZIP Code              0
Family                0
CCAvg                 0
Education             0
Mortgage              0
Personal Loan         0
Securities Account    0
CD Account            0
Online                0
CreditCard            0
dtype: int64

In [113]:
data.dtypes

ID                      int64
Age                     int64
Experience              int64
Income                  int64
ZIP Code                int64
Family                  int64
CCAvg                 float64
Education               int64
Mortgage                int64
Personal Loan           int64
Securities Account      int64
CD Account              int64
Online                  int64
CreditCard              int64
dtype: object

In [211]:
import pandas as pd

# Count the occurrences of each class label in the resampled training and testing sets
train_balance = pd.Series(y_train).value_counts()
test_balance = pd.Series(y_test).value_counts()

# Print the class distribution
print("Class distribution in the original training set:")
print(train_balance)

print("\nClass distribution in the original testing set:")
print(test_balance)

Class distribution in the original training set:
Personal Loan
0    3625
1     375
Name: count, dtype: int64

Class distribution in the original testing set:
Personal Loan
0    895
1    105
Name: count, dtype: int64


In [213]:
data.dropna(inplace=True)

# Encoding object columns
le = LabelEncoder()
data['Personal Loan'] = le.fit_transform(data['Personal Loan'])

# Splitting features and labels
X = data.drop('Personal Loan', axis=1)
y = data['Personal Loan']

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## BASELINE CLASSIFICATION ALGORITHMS WITH IMBALANCED DATASET

### KNN

In [171]:
from sklearn.model_selection import cross_val_score

def knn_classifier(X, y, cv=5):
    # Initialize KNN classifier
    knn = KNeighborsClassifier()
    
    # Standardizing the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Perform cross-validation
    accuracy_scores = cross_val_score(knn, X_scaled, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(knn, X_scaled, y, cv=cv, scoring='precision')
    recall_scores = cross_val_score(knn, X_scaled, y, cv=cv, scoring='recall')
    f1_scores = cross_val_score(knn, X_scaled, y, cv=cv, scoring='f1')
    
    # Compute mean scores
    accuracy = accuracy_scores.mean()
    precision = precision_scores.mean()
    recall = recall_scores.mean()
    f1 = f1_scores.mean()
    
    return accuracy, precision, recall, f1

# Example usage:
accuracy, precision, recall, f1 = knn_classifier(X_train, y_train)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.9470000000000001
Precision: 0.8976489179387948
Recall: 0.49066666666666664
F1 Score: 0.6340481637633257


In [93]:
def correlation_method(X_train, X_test, threshold):
    corr_matrix = X_train.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    X_train_selected = X_train.drop(to_drop, axis=1)
    X_test_selected = X_test.drop(to_drop, axis=1)
    
    return X_train_selected, X_test_selected
X_train_corr, X_test_corr = correlation_method(X_train, X_test, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = knn_classifier(X_train_corr, X_test_corr, y_train, y_test)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

Accuracy with Correlation method: 0.951
Precision with Correlation method: 0.9242424242424242
Recall with Correlation method: 0.580952380952381
F1 Score with Correlation method: 0.7134502923976608


In [137]:
def mutual_information(X_train, y_train, X_test, n_features):
    selector = SelectKBest(score_func=mutual_info_classif, k=n_features)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)
    return X_train_selected, X_test_selected

X_train_mi, X_test_mi = mutual_information(X_train,y_train, X_test, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = knn_classifier(X_train_mi, y_train)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

Accuracy with Mutual Information method: 0.95225
Precision with Mutual Information method: 0.9430094815081688
Recall with Mutual Information method: 0.5226666666666666
F1 Score with Mutual Information method: 0.6697328995078526


In [95]:
def covariance_method(X_train, X_test, threshold):
    cov_matrix = X_train.cov().abs()
    upper = cov_matrix.where(np.triu(np.ones(cov_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    X_train_selected = X_train.drop(to_drop, axis=1)
    X_test_selected = X_test.drop(to_drop, axis=1)
    
    return X_train_selected, X_test_selected

# Example usage:
X_train_cov, X_test_cov = covariance_method(X_train, X_test, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = knn_classifier(X_train_cov, X_test_cov, y_train, y_test)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)


Accuracy with Covariance method: 0.898
Precision with Covariance method: 0.8
Recall with Covariance method: 0.0380952380952381
F1 Score with Covariance method: 0.07272727272727274


In [17]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# KNN Classifier
def knn_classifier(X_train, X_test, y_train, y_test):
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    return accuracy, precision, recall, f1

# Backward Feature Elimination (Custom)
def backward_feature_elimination_knn(X_train, X_test, y_train, y_test, n_features_to_select):
    X_train = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
    X_test = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
    
    remaining_features = list(range(X_train.shape[1]))
    selected_features = list(range(X_train.shape[1]))
    best_accuracy = knn_classifier(X_train, X_test, y_train, y_test)[0] # Initial accuracy
    
    while len(selected_features) > n_features_to_select:
        worst_feature = None
        for feature in selected_features:
            features_to_use = selected_features.copy()
            features_to_use.remove(feature)
            X_train_temp = X_train[:, features_to_use]
            X_test_temp = X_test[:, features_to_use]
            
            accuracy, _, _, _ = knn_classifier(X_train_temp, X_test_temp, y_train, y_test)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                worst_feature = feature
        
        if worst_feature is not None:
            selected_features.remove(worst_feature)
        else:
            break
    
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]
    
    return X_train_selected, X_test_selected
def forward_feature_selection_knn(X_train, X_test, y_train, y_test, n_features_to_select):
    X_train = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
    X_test = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
    
    remaining_features = list(range(X_train.shape[1]))
    selected_features = []
    
    while len(selected_features) < n_features_to_select:
        best_accuracy = 0
        best_feature = None
        for feature in remaining_features:
            features_to_use = selected_features + [feature]
            X_train_temp = X_train[:, features_to_use]
            X_test_temp = X_test[:, features_to_use]
            
            accuracy, _, _, _ = knn_classifier(X_train_temp, X_test_temp, y_train, y_test)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_feature = feature
        
        if best_feature is not None:
            selected_features.append(best_feature)
            remaining_features.remove(best_feature)
        else:
            break
    
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]
    
    return X_train_selected, X_test_selected
def recursive_feature_elimination_knn(X_train, X_test, y_train, y_test, n_features_to_select):
    X_train = X_train.values if isinstance(X_train, pd.DataFrame) else X_train
    X_test = X_test.values if isinstance(X_test, pd.DataFrame) else X_test
    
    remaining_features = list(range(X_train.shape[1]))
    selected_features = list(range(X_train.shape[1]))
    best_accuracy = knn_classifier(X_train, X_test, y_train, y_test)[0] # Initial accuracy
    
    while len(selected_features) > n_features_to_select:
        worst_feature = None
        for feature in selected_features:
            features_to_use = selected_features.copy()
            features_to_use.remove(feature)
            X_train_temp = X_train[:, features_to_use]
            X_test_temp = X_test[:, features_to_use]
            
            accuracy, _, _, _ = knn_classifier(X_train_temp, X_test_temp, y_train, y_test)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                worst_feature = feature
        
        if worst_feature is not None:
            selected_features.remove(worst_feature)
        else:
            break
    
    X_train_selected = X_train[:, selected_features]
    X_test_selected = X_test[:, selected_features]
    
    return X_train_selected, X_test_selected
# Wrapper Methods
def wrapper_feature_selection(method, X_train, X_test, y_train, y_test, n_features_to_select):
    if method == 'backward':
        return backward_feature_elimination_knn(X_train, X_test, y_train, y_test, n_features_to_select)
    elif method == 'forward':
        return forward_feature_selection_knn(X_train, X_test, y_train, y_test, n_features_to_select)
    elif method == 'recursive':
        return recursive_feature_elimination_knn(X_train, X_test, y_train, y_test, n_features_to_select)
    else:
        raise ValueError("Invalid method. Choose 'backward', 'forward', or 'recursive'.")

# Example usage:
methods = ['backward', 'forward', 'recursive']

for method in methods:
    X_train_selected, X_test_selected = wrapper_feature_selection(method, X_train, X_test, y_train, y_test, n_features_to_select=5)
    accuracy, precision, recall, f1 = knn_classifier(X_train_selected, X_test_selected, y_train, y_test)
    print(f"Method: {method.capitalize()} Feature Selection")
    print("Accuracy:", accuracy)
    print("Precision:", precision)
    print("Recall:", recall)
    print("F1 Score:", f1)
    print()


Method: Backward Feature Selection
Accuracy: 0.935
Precision: 0.9362471472042603
Recall: 0.935
F1 Score: 0.9227656735521159



C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetri

Method: Forward Feature Selection
Accuracy: 0.96
Precision: 0.9588496011031222
Recall: 0.96
F1 Score: 0.9572311077261573

Method: Recursive Feature Selection
Accuracy: 0.935
Precision: 0.9362471472042603
Recall: 0.935
F1 Score: 0.9227656735521159



In [18]:
from sklearn.decomposition import PCA

# PCA Dimensionality Reduction
def perform_pca(X_train, X_test, n_components):
    pca = PCA(n_components=n_components)
    pca.fit(X_train)
    X_train_pca = pca.transform(X_train)
    X_test_pca = pca.transform(X_test)
    return X_train_pca, X_test_pca

# Main function
def main(X_train, X_test, y_train, y_test):
    # Perform PCA
    n_components = 10  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train, X_test, n_components)
    
    # Evaluate KNN classifier using PCA
    accuracy, precision, recall, f1 = knn_classifier(X_train_pca, X_test_pca, y_train, y_test)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train, X_test, y_train, y_test)  # Replace X_train, X_test, y_train, y_test with your preprocessed data


Accuracy with PCA: 0.896
Precision with PCA: 0.866251256281407
Recall with PCA: 0.896
F1 Score with PCA: 0.8514785954785956


### LOGISTIC REGRESSION

In [73]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def logistic_regression_classifier(X, y, cv=5):
    # Standardizing the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Initialize Logistic Regression classifier
    logistic_reg = LogisticRegression()
    
    # Perform cross-validation
    accuracy_scores = cross_val_score(logistic_reg, X_scaled, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(logistic_reg, X_scaled, y, cv=cv, scoring='precision')
    recall_scores = cross_val_score(logistic_reg, X_scaled, y, cv=cv, scoring='recall')
    f1_scores = cross_val_score(logistic_reg, X_scaled, y, cv=cv, scoring='f1')
    
    # Compute mean scores
    accuracy = accuracy_scores.mean()
    precision = precision_scores.mean()
    recall = recall_scores.mean()
    f1 = f1_scores.mean()
    
    return accuracy, precision, recall, f1

# Example usage:
accuracy, precision, recall, f1 = logistic_regression_classifier(X_train, y_train)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.9497500000000001
Precision: 0.8194257010046483
Recall: 0.5946666666666667
F1 Score: 0.6889014665978592


In [74]:
X_train_corr, X_test_corr = correlation_method(X_train, X_test, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = logistic_regression_classifier(X_train_corr, y_train)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

Accuracy with Correlation method: 0.95
Precision with Correlation method: 0.8221049103402045
Recall with Correlation method: 0.5946666666666667
F1 Score with Correlation method: 0.6897820383445726


In [138]:
X_train_mi, X_test_mi = mutual_information(X_train, y_train, X_test, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = logistic_regression_classifier(X_train_mi, y_train)

print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

Accuracy with Mutual Information method: 0.9465
Precision with Mutual Information method: 0.8054102716114441
Recall with Mutual Information method: 0.5653333333333332
F1 Score with Mutual Information method: 0.6634873323397914


In [76]:
X_train_cov, X_test_cov = covariance_method(X_train, X_test, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = logistic_regression_classifier(X_train_cov, y_train)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

Accuracy with Covariance method: 0.90625
Precision with Covariance method: 0.0
Recall with Covariance method: 0.0
F1 Score with Covariance method: 0.0


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

In [77]:
def main(X_train, X_test, y_train, y_test):
    # Perform PCA
    n_components = 10  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train, X_test, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = logistic_regression_classifier(X_train_pca, y_train)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train, X_test, y_train, y_test)

Accuracy with PCA: 0.94275
Precision with PCA: 0.780455872076294
Recall with PCA: 0.5413333333333334
F1 Score with PCA: 0.6389712419757088


In [78]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.base import clone

def forward_feature_selection(X, y, model):
    selected_features = []
    best_score = 0
    
    while True:
        remaining_features = [feature for feature in X.columns if feature not in selected_features]
        if not remaining_features:
            break
        
        best_feature = None
        for feature in remaining_features:
            features_to_try = selected_features + [feature]
            X_subset = X[features_to_try]
            scores = cross_val_score(model, X_subset, y, cv=5)
            mean_score = scores.mean()
            if mean_score > best_score:
                best_score = mean_score
                best_feature = feature
        
        if best_feature:
            selected_features.append(best_feature)
        else:
            break
    
    return selected_features

# Apply forward feature selection
selected_features_forward = forward_feature_selection(X_train, y_train, LogisticRegression())


# Selecting features based on the selected features
X_train_forward = X_train[selected_features_forward]
X_test_forward = X_test[selected_features_forward]

# Apply KNN classifier with selected features
accuracy, precision, recall, f1 = logistic_regression_classifier(X_train_forward, y_train)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

Accuracy: 0.95175
Precision: 0.833517440169183
Recall: 0.608
F1 Score: 0.7027705769323892


In [79]:
def backward_feature_selection(X, y, model):
    selected_features = list(X.columns)
    best_score = cross_val_score(model, X, y, cv=5).mean()
    
    while True:
        removed_feature = None
        for feature in selected_features:
            features_to_try = [f for f in selected_features if f != feature]
            X_subset = X[features_to_try]
            scores = cross_val_score(model, X_subset, y, cv=5)
            mean_score = scores.mean()
            if mean_score > best_score:
                best_score = mean_score
                removed_feature = feature
        
        if removed_feature:
            selected_features.remove(removed_feature)
        else:
            break
    
    return selected_features

# Apply forward feature selection
selected_features_backward = backward_feature_selection(X_train, y_train, LogisticRegression())

# Selecting features based on the selected features
X_train_backward = X_train[selected_features_backward]
X_test_backward = X_test[selected_features_backward]

# Apply KNN classifier with selected features
accuracy, precision, recall, f1 = logistic_regression_classifier(X_train_backward, y_train)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i 

Accuracy: 0.951
Precision: 0.8187605801623954
Recall: 0.6133333333333334
F1 Score: 0.7008115645816382


### RANDOM FOREST

In [180]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

def random_forest_classifier(X, y, cv=5):
    # Initialize Random Forest classifier
    rf_classifier = RandomForestClassifier(random_state=42)
    
    # Perform cross-validation
    accuracy_scores = cross_val_score(rf_classifier, X, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(rf_classifier, X, y, cv=cv, scoring='precision')
    recall_scores = cross_val_score(rf_classifier, X, y, cv=cv, scoring='recall')
    f1_scores = cross_val_score(rf_classifier, X, y, cv=cv, scoring='f1')
    
    # Compute mean scores
    accuracy = accuracy_scores.mean()
    precision = precision_scores.mean()
    recall = recall_scores.mean()
    f1 = f1_scores.mean()
    
    return accuracy, precision, recall, f1

# Assuming you have already split your data into X_train, X_test, y_train, y_test
accuracy, precision, recall, f1 = random_forest_classifier(X_train, y_train)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9845
Precision: 0.9845572972778855
Recall: 0.8480000000000001
F1 Score: 0.9110058370364907


In [65]:
accuracy_corr, precision_corr, recall_corr, f1_corr = random_forest_classifier(X_train_corr, y_train)

# Print the results
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

Accuracy with Correlation method: 0.98375
Precision with Correlation method: 0.9724866184285197
Recall with Correlation method: 0.8506666666666668
F1 Score with Correlation method: 0.9071062462545916


In [139]:
X_train_mi, X_test_mi = mutual_information(X_train, y_train, X_test, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = random_forest_classifier(X_train_mi, y_train)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

Accuracy with Mutual Information method: 0.96975
Precision with Mutual Information method: 0.9259499642459875
Recall with Mutual Information method: 0.736
F1 Score with Mutual Information method: 0.8196263552127542


In [67]:
X_train_cov, X_test_cov = covariance_method(X_train, X_test, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = random_forest_classifier(X_train_cov, y_train)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

Accuracy with Covariance method: 0.836
Precision with Covariance method: 0.10316800633296797
Recall with Covariance method: 0.09866666666666667
F1 Score with Covariance method: 0.1007469017202054


In [68]:
def main(X_train, X_test, y_train, y_test):
    # Perform PCA
    n_components = 10  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train, X_test, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = random_forest_classifier(X_train_pca, y_train)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train, X_test, y_train, y_test)

Accuracy with PCA: 0.9785
Precision with PCA: 0.9647670250896058
Recall with PCA: 0.7999999999999999
F1 Score with PCA: 0.8746138204215004


In [71]:
# Step 2: Call the forward feature selection function with your training data
selected_features_forward = forward_feature_selection(X_train, y_train, RandomForestClassifier())

# Step 3: Use the selected features to subset your training and testing data
X_train_forward = X_train[selected_features_forward]
X_test_forward = X_test[selected_features_forward]

# Step 4: Apply your random_forest_classifier function with the subsetted data
accuracy, precision, recall, f1 = random_forest_classifier(X_train_forward, y_train)

# Print the results
print("Accuracy With Forward Feature Selection:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy with Random Forest and Forward Feature Selection:
Accuracy: 0.9865
Precision: 0.9793010253049328
Recall: 0.8746666666666666
F1 Score: 0.9237249367227978


In [72]:
# Apply forward feature selection
selected_features_backward = backward_feature_selection(X_train, y_train, RandomForestClassifier())

# Selecting features based on the selected features
X_train_backward = X_train[selected_features_backward]
X_test_backward = X_test[selected_features_backward]

# Apply KNN classifier with selected features
accuracy, precision, recall, f1 = random_forest_classifier(X_train_backward, y_train)

# Print the result
print("Accuracy With Backward Feature Selection:")
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.98575
Precision: 0.988054376944708
Recall: 0.8586666666666666
F1 Score: 0.9184290498717136


### SVM

In [185]:
from sklearn.svm import SVC, LinearSVC
from sklearn.feature_selection import RFE
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score

def svm_classifier(X, y, cv=5):
    # Initialize SVM classifier
    svm_classifier = SVC(kernel='rbf', random_state=42)
    
    # Perform cross-validation
    accuracy_scores = cross_val_score(svm_classifier, X, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(svm_classifier, X, y, cv=cv, scoring='precision')
    recall_scores = cross_val_score(svm_classifier, X, y, cv=cv, scoring='recall')
    f1_scores = cross_val_score(svm_classifier, X, y, cv=cv, scoring='f1')
    
    # Compute mean scores
    accuracy = accuracy_scores.mean()
    precision = precision_scores.mean()
    recall = recall_scores.mean()
    f1 = f1_scores.mean()
    
    return accuracy, precision, recall, f1

# Example usage:
accuracy, precision, recall, f1 = svm_classifier(X_train, y_train)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

Accuracy: 0.90625
Precision: 0.0
Recall: 0.0
F1 Score: 0.0


In [214]:
X_train_corr, X_test_corr = correlation_method(X_train, X_test, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = svm_classifier(X_train_corr, y_train)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train, y_train, X_test, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = svm_classifier(X_train_mi, y_train)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train, X_test, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = svm_classifier(X_train_cov, y_train)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train, X_test, y_train, y_test):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train, X_test, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = svm_classifier(X_train_pca, y_train)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train, X_test, y_train, y_test)

# Apply forward feature selection
selected_features_forward = backward_feature_selection(X_train, y_train, SVC())


# Selecting features based on the selected features
X_train_forward = X_train[selected_features_forward]
X_test_forward = X_test[selected_features_forward]

# Apply KNN classifier with selected features
accuracy, precision, recall, f1 = svm_classifier(X_train_forward, y_train)

# Print the results
print("Accuracy with forward selection:", accuracy)
print("Precision with forward selection:", precision)
print("Recall with forward selection:", recall)
print("F1 Score: with forward selection", f1)


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

Accuracy with Correlation method: 0.90625
Precision with Correlation method: 0.0
Recall with Correlation method: 0.0
F1 Score with Correlation method: 0.0


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

Accuracy with Mutual Information method: 0.90625
Precision with Mutual Information method: 0.0
Recall with Mutual Information method: 0.0
F1 Score with Mutual Information method: 0.0


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

Accuracy with Covariance method: 0.90625
Precision with Covariance method: 0.0
Recall with Covariance method: 0.0
F1 Score with Covariance method: 0.0


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

Accuracy with PCA: 0.90625
Precision with PCA: 0.0
Recall with PCA: 0.0
F1 Score with PCA: 0.0


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

Accuracy with forward selection: 0.90625
Precision with forward selection: 0.0
Recall with forward selection: 0.0
F1 Score: with forward selection 0.0


### NAIVE BAYES

In [187]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score
from statistics import mean

def naive_bayes_classifier(X, y, cv=5):
    # Initialize Naive Bayes classifier
    nb_classifier = GaussianNB()
    
    # Perform cross-validation
    accuracy_scores = cross_val_score(nb_classifier, X, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(nb_classifier, X, y, cv=cv, scoring='precision')
    recall_scores = cross_val_score(nb_classifier, X, y, cv=cv, scoring='recall')
    f1_scores = cross_val_score(nb_classifier, X, y, cv=cv, scoring='f1')
    
    # Compute mean scores
    accuracy = mean(accuracy_scores)
    precision = mean(precision_scores)
    recall = mean(recall_scores)
    f1 = mean(f1_scores)
    
    return accuracy, precision, recall, f1

# Example usage:
accuracy, precision, recall, f1 = naive_bayes_classifier(X_train, y_train)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.882
Precision: 0.40762095882804183
Recall: 0.56
F1 Score: 0.47130334991929856


In [106]:
X_train_corr, X_test_corr = correlation_method(X_train, X_test, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = naive_bayes_classifier(X_train_corr, y_train)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train, y_train, X_test, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = naive_bayes_classifier(X_train_mi, y_train)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train, X_test, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = naive_bayes_classifier(X_train_cov, y_train)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train, X_test, y_train, y_test):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train, X_test, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = naive_bayes_classifier(X_train_pca, y_train)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train, X_test, y_train, y_test)

# Apply forward feature selection
selected_features_forward = backward_feature_selection(X_train, y_train, GaussianNB())


# Selecting features based on the selected features
X_train_forward = X_train[selected_features_forward]
X_test_forward = X_test[selected_features_forward]

# Apply KNN classifier with selected features
accuracy, precision, recall, f1 = naive_bayes_classifier(X_train_forward, y_train)

# Print the results
print("Accuracy with forward selection:", accuracy)
print("Precision with forward selection:", precision)
print("Recall with forward selection:", recall)
print("F1 Score: with forward selection", f1)

# Initialize LinearSVC instead of SVC
selected_features_backward = backward_feature_selection(X_train, y_train, GaussianNB())

# Selecting features based on the selected features
X_train_backward = X_train[selected_features_backward]
X_test_backward = X_test[selected_features_backward]

# Apply KNN classifier with selected features
accuracy, precision, recall, f1 = naive_bayes_classifier(X_train_backward, y_train)

# Print the results
print("Accuracy with backward selection:", accuracy)
print("Precision  with backward selection:", precision)
print("Recall with backward selection:", recall)
print("F1 Score with backward selection:", f1)

Accuracy with Correlation method: 0.88175
Precision with Correlation method: 0.4064888833563437
Recall with Correlation method: 0.5573333333333333
F1 Score with Correlation method: 0.4696090589432397
Accuracy with Mutual Information method: 0.882
Precision with Mutual Information method: 0.40802278880948917
Recall with Mutual Information method: 0.5653333333333334
F1 Score with Mutual Information method: 0.47361486112995504


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

Accuracy with Covariance method: 0.90625
Precision with Covariance method: 0.0
Recall with Covariance method: 0.0
F1 Score with Covariance method: 0.0


C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DELL\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is i

Accuracy with PCA: 0.906
Precision with PCA: 0.0
Recall with PCA: 0.0
F1 Score with PCA: 0.0
Accuracy with forward selection: 0.92575
Precision with forward selection: 0.6490468929290051
Recall with forward selection: 0.4693333333333333
F1 Score: with forward selection 0.5430956126528634
Accuracy with backward selection: 0.92575
Precision  with backward selection: 0.6490468929290051
Recall with backward selection: 0.4693333333333333
F1 Score with backward selection: 0.5430956126528634


### DECISION TREE

In [188]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from statistics import mean

def decision_tree_classifier(X, y, cv=5):
    # Initialize Decision Tree classifier
    dt_classifier = DecisionTreeClassifier(random_state=42)
    
    # Perform cross-validation
    accuracy_scores = cross_val_score(dt_classifier, X, y, cv=cv, scoring='accuracy')
    precision_scores = cross_val_score(dt_classifier, X, y, cv=cv, scoring='precision')
    recall_scores = cross_val_score(dt_classifier, X, y, cv=cv, scoring='recall')
    f1_scores = cross_val_score(dt_classifier, X, y, cv=cv, scoring='f1')
    
    # Compute mean scores
    accuracy = mean(accuracy_scores)
    precision = mean(precision_scores)
    recall = mean(recall_scores)
    f1 = mean(f1_scores)
    
    return accuracy, precision, recall, f1

# Example usage:
accuracy, precision, recall, f1 = decision_tree_classifier(X_train, y_train)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.97775
Precision: 0.8779993180335646
Recall: 0.8853333333333333
F1 Score: 0.8813813303966105


In [109]:
X_train_corr, X_test_corr = correlation_method(X_train, X_test, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = decision_tree_classifier(X_train_corr, y_train)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train, y_train, X_test, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = decision_tree_classifier(X_train_mi, y_train)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train, X_test, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = decision_tree_classifier(X_train_cov, y_train)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train, X_test, y_train, y_test):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train, X_test, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = decision_tree_classifier(X_train_pca, y_train)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train, X_test, y_train, y_test)

# Apply forward feature selection
selected_features_forward = backward_feature_selection(X_train, y_train, GaussianNB())


# Selecting features based on the selected features
X_train_forward = X_train[selected_features_forward]
X_test_forward = X_test[selected_features_forward]

# Apply KNN classifier with selected features
accuracy, precision, recall, f1 = decision_tree_classifier(X_train_forward, y_train)

# Print the results
print("Accuracy with forward selection:", accuracy)
print("Precision with forward selection:", precision)
print("Recall with forward selection:", recall)
print("F1 Score: with forward selection", f1)

# Initialize LinearSVC instead of SVC
selected_features_backward = backward_feature_selection(X_train, y_train, GaussianNB())

# Selecting features based on the selected features
X_train_backward = X_train[selected_features_backward]
X_test_backward = X_test[selected_features_backward]

# Apply KNN classifier with selected features
accuracy, precision, recall, f1 = decision_tree_classifier(X_train_backward, y_train)

# Print the results
print("Accuracy with backward selection:", accuracy)
print("Precision  with backward selection:", precision)
print("Recall with backward selection:", recall)
print("F1 Score with backward selection:", f1)

Accuracy with Correlation method: 0.977
Precision with Correlation method: 0.8696538461538461
Recall with Correlation method: 0.888
F1 Score with Correlation method: 0.8785764284208306
Accuracy with Mutual Information method: 0.97975
Precision with Mutual Information method: 0.9014195154058168
Recall with Mutual Information method: 0.88
F1 Score with Mutual Information method: 0.8905248423330878
Accuracy with Covariance method: 0.836
Precision with Covariance method: 0.10316800633296798
Recall with Covariance method: 0.09866666666666667
F1 Score with Covariance method: 0.1007469017202054
Accuracy with PCA: 0.826
Precision with PCA: 0.09341087190971228
Recall with PCA: 0.10133333333333333
F1 Score with PCA: 0.0969484057209535
Accuracy with forward selection: 0.97
Precision with forward selection: 0.8343436447536202
Recall with forward selection: 0.8506666666666667
F1 Score: with forward selection 0.8414800701461844
Accuracy with backward selection: 0.97
Precision  with backward selectio

## 1- BALANCING TECHNIQUE: SMOTE

### SAMPLING

In [118]:
from imblearn.over_sampling import SMOTE

# Apply SMOTE to the entire dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Split the resampled data into training and testing sets
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

import pandas as pd

# Count the occurrences of each class label in the resampled training and testing sets
train_balance = pd.Series(y_train_s).value_counts()
test_balance = pd.Series(y_test_s).value_counts()

# Print the class distribution
print("Class distribution in the resampled training set:")
print(train_balance)

print("\nClass distribution in the resampled testing set:")
print(test_balance)

import pandas as pd

# Create a DataFrame for the resampled data
resampled_data = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)

# Define the column names
column_names = list(X.columns) + ['Personal Loan']

# Set the column names
resampled_data.columns = column_names

# Save the resampled data to a CSV file
resampled_data.to_excel('Bank_loan_SMOTE.xlsx', index=False)


Class distribution in the resampled training set:
Personal Loan
1    3645
0    3587
Name: count, dtype: int64

Class distribution in the resampled testing set:
Personal Loan
0    933
1    875
Name: count, dtype: int64


### KNN

In [127]:
accuracy, precision, recall, f1 = knn_classifier(X_train_s, y_train_s)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9409561060657765
Precision: 0.9161504898501981
Recall: 0.9720164609053498
F1 Score: 0.9432055275916988


In [215]:
X_train_corr, X_test_corr = correlation_method(X_train_s, X_test_s, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = knn_classifier(X_train_corr, y_train_s)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_s, y_train_s, X_test_s, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = knn_classifier(X_train_mi, y_train_s)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_s, X_test_s, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = knn_classifier(X_train_cov, y_train_s)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_s, X_test_s, y_train_s, y_test_s):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_s, X_test_s, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = knn_classifier(X_train_pca, y_train_s)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_s, X_test_s, y_train_s, y_test_s)

Accuracy with Correlation method: 0.942615570345858
Precision with Correlation method: 0.9197717727434569
Recall with Correlation method: 0.97119341563786
F1 Score with Correlation method: 0.9446890881992026
Accuracy with Mutual Information method: 0.9398495097884594
Precision with Mutual Information method: 0.9285349969733236
Recall with Mutual Information method: 0.9541838134430727
F1 Score with Mutual Information method: 0.9411499664837806
Accuracy with Covariance method: 0.48257806249587787
Precision with Covariance method: 0.48739048199192636
Recall with Covariance method: 0.522085048010974
F1 Score with Covariance method: 0.5040877353815769
Accuracy with PCA: 0.7256624809664867
Precision with PCA: 0.6968633433385436
Recall with PCA: 0.8065843621399177
F1 Score with PCA: 0.7476409957307111


### LOGISTIC REGRESSION

In [142]:
accuracy, precision, recall, f1 = logistic_regression_classifier(X_train_s, y_train_s)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9012712905319443
Precision: 0.8925391768752761
Recall: 0.91440329218107
F1 Score: 0.9031933431396896


In [143]:
X_train_corr, X_test_corr = correlation_method(X_train_s, X_test_s, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = logistic_regression_classifier(X_train_corr, y_train_s)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_s, y_train_s, X_test_s, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = logistic_regression_classifier(X_train_mi, y_train_s)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_s, X_test_s, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = logistic_regression_classifier(X_train_cov, y_train_s)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_s, X_test_s, y_train_s, y_test_s):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_s, X_test_s, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = logistic_regression_classifier(X_train_pca, y_train_s)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_s, X_test_s, y_train_s, y_test_s)

Accuracy with Correlation method: 0.9008564483583624
Precision with Correlation method: 0.8916208387671698
Recall with Correlation method: 0.9146776406035666
F1 Score with Correlation method: 0.9028547955378595
Accuracy with Mutual Information method: 0.8893804226993225
Precision with Mutual Information method: 0.8785904529940824
Recall with Mutual Information method: 0.9058984910836763
F1 Score with Mutual Information method: 0.8919312691961684
Accuracy with Covariance method: 0.518943662712284
Precision with Covariance method: 0.5199450562620462
Recall with Covariance method: 0.594238683127572
F1 Score with Covariance method: 0.5545574029891807
Accuracy with PCA: 0.5196351300587565
Precision with PCA: 0.5207062328708278
Recall with PCA: 0.5928669410150892
F1 Score with PCA: 0.5543695969123027


### RANDOM FOREST

In [144]:
accuracy, precision, recall, f1 = random_forest_classifier(X_train_s, y_train_s)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9781517729723633
Precision: 0.9770336062151463
Recall: 0.9796982167352539
F1 Score: 0.9783548660931569


In [145]:
X_train_corr, X_test_corr = correlation_method(X_train_s, X_test_s, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = random_forest_classifier(X_train_corr, y_train_s)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_s, y_train_s, X_test_s, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = random_forest_classifier(X_train_mi, y_train_s)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_s, X_test_s, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = random_forest_classifier(X_train_cov, y_train_s)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_s, X_test_s, y_train_s, y_test_s):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_s, X_test_s, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = random_forest_classifier(X_train_pca, y_train_s)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_s, X_test_s, y_train_s, y_test_s)

Accuracy with Correlation method: 0.977045845795326
Precision with Correlation method: 0.9759430927274518
Recall with Correlation method: 0.9786008230452674
F1 Score with Correlation method: 0.9772642731040333
Accuracy with Mutual Information method: 0.9752482601003077
Precision with Mutual Information method: 0.9763800373477609
Recall with Mutual Information method: 0.974485596707819
F1 Score with Mutual Information method: 0.9754225075818663
Accuracy with Covariance method: 0.42795969339913453
Precision with Covariance method: 0.4392314187402036
Recall with Covariance method: 0.48806584362139915
F1 Score with Covariance method: 0.4623183320273835
Accuracy with PCA: 0.7679758091573065
Precision with PCA: 0.7608647704366149
Recall with PCA: 0.7871056241426612
F1 Score with PCA: 0.7736896661675245


### SVM

In [150]:
accuracy, precision, recall, f1 = svm_classifier(X_train_s, y_train_s)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


Accuracy: 0.5040100135636185
Precision: 0.5040100135636185
Recall: 1.0
F1 Score: 0.6702216024340769


In [156]:
X_train_corr, X_test_corr = correlation_method(X_train_s, X_test_s, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = svm_classifier(X_train_corr, y_train_s)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_s, y_train_s, X_test_s, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = svm_classifier(X_train_mi, y_train_s)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_s, X_test_s, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = svm_classifier(X_train_cov, y_train_s)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_s, X_test_s, y_train_s, y_test_s):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_s, X_test_s, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = svm_classifier(X_train_pca, y_train_s)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_s, X_test_s, y_train_s, y_test_s)

Accuracy with Correlation method: 0.5040100135636185
Precision with Correlation method: 0.5040100135636185
Recall with Correlation method: 1.0
F1 Score with Correlation method: 0.6702216024340769
Accuracy with Mutual Information method: 0.5040100135636185
Precision with Mutual Information method: 0.5040100135636185
Recall with Mutual Information method: 1.0
F1 Score with Mutual Information method: 0.6702216024340769
Accuracy with Covariance method: 0.5141035824584848
Precision with Covariance method: 0.5142128059534382
Recall with Covariance method: 0.6378600823045268
F1 Score with Covariance method: 0.5642043292568875
Accuracy with PCA: 0.5334608447295448
Precision with PCA: 0.5338701512851114
Recall with PCA: 0.5945130315500686
F1 Score with PCA: 0.5619011890009602


### NAIVE BAYES

In [151]:
accuracy, precision, recall, f1 = naive_bayes_classifier(X_train_s, y_train_s)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8940801830658366
Precision: 0.8820494317841793
Recall: 0.9119341563786009
F1 Score: 0.8966292389994871


In [154]:
X_train_corr, X_test_corr = correlation_method(X_train_s, X_test_s, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = naive_bayes_classifier(X_train_corr, y_train_s)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_s, y_train_s, X_test_s, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = naive_bayes_classifier(X_train_mi, y_train_s)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_s, X_test_s, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = naive_bayes_classifier(X_train_cov, y_train_s)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_s, X_test_s, y_train_s, y_test_s):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_s, X_test_s, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = naive_bayes_classifier(X_train_pca, y_train_s)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_s, X_test_s, y_train_s, y_test_s)

Accuracy with Correlation method: 0.8946330510686009
Precision with Correlation method: 0.8825560995990287
Recall with Correlation method: 0.912482853223594
F1 Score with Correlation method: 0.8971649296483181
Accuracy with Mutual Information method: 0.8921454318134243
Precision with Mutual Information method: 0.8810269278831063
Recall with Mutual Information method: 0.9089163237311385
F1 Score with Mutual Information method: 0.8946748912513425
Accuracy with Covariance method: 0.514796292419763
Precision with Covariance method: 0.5140974246138931
Recall with Covariance method: 0.6820301783264746
F1 Score with Covariance method: 0.5861599592670141
Accuracy with PCA: 0.5174240403907163
Precision with PCA: 0.5153011626103785
Recall with PCA: 0.7286694101508916
F1 Score with PCA: 0.6033715246604803


### DECISION TREE

In [152]:
accuracy, precision, recall, f1 = decision_tree_classifier(X_train_s, y_train_s)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9589305292296457
Precision: 0.956865098183407
Recall: 0.9618655692729767
F1 Score: 0.9593537433856784


In [155]:
X_train_corr, X_test_corr = correlation_method(X_train_s, X_test_s, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = decision_tree_classifier(X_train_corr, y_train_s)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_s, y_train_s, X_test_s, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = decision_tree_classifier(X_train_mi, y_train_s)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_s, X_test_s, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = decision_tree_classifier(X_train_cov, y_train_s)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_s, X_test_s, y_train_s, y_test_s):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_s, X_test_s, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = decision_tree_classifier(X_train_pca, y_train_s)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_s, X_test_s, y_train_s, y_test_s)

Accuracy with Correlation method: 0.9594832060609015
Precision with Correlation method: 0.9571893611963382
Recall with Correlation method: 0.9626886145404664
F1 Score with Correlation method: 0.9599171278154656
Accuracy with Mutual Information method: 0.9608668098541266
Precision with Mutual Information method: 0.9610614594774483
Recall with Mutual Information method: 0.9613168724279836
F1 Score with Mutual Information method: 0.961185284812303
Accuracy with Covariance method: 0.39933759072282904
Precision with Covariance method: 0.39863962712047063
Recall with Covariance method: 0.37777777777777777
F1 Score with Covariance method: 0.3878379398580697
Accuracy with PCA: 0.7441930220487659
Precision with PCA: 0.7396823205017491
Recall with PCA: 0.7604938271604939
F1 Score with PCA: 0.7496650876996431


## 2- BALANCING TECHNIQUE: CLUSTER BASED OVERSAMPLING

### SAMPLING

In [164]:
from imblearn.over_sampling import KMeansSMOTE

# Adjust parameters
kmeans_smote = KMeansSMOTE(cluster_balance_threshold=0.1, k_neighbors=10, random_state=42)

# Apply K-means SMOTE for cluster-based oversampling
X_resampled, y_resampled = kmeans_smote.fit_resample(X, y)

X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Count the occurrences of each class label in the resampled training and testing sets
train_balance = pd.Series(y_train_c).value_counts()
test_balance = pd.Series(y_test_c).value_counts()

# Print the class distribution
print("Class distribution in the resampled training set:")
print(train_balance)

print("\nClass distribution in the resampled testing set:")
print(test_balance)

import pandas as pd

# Create a DataFrame for the resampled data
resampled_data = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)

# Define the column names
column_names = list(X.columns) + ['Personal Loan']

# Set the column names
resampled_data.columns = column_names

# Save the resampled data to a CSV file
resampled_data.to_excel('Bank_loan_CLUSTER.xlsx', index=False)

Class distribution in the resampled training set:
Personal Loan
1    3623
0    3610
Name: count, dtype: int64

Class distribution in the resampled testing set:
Personal Loan
0    910
1    899
Name: count, dtype: int64


### KNN

In [166]:
accuracy, precision, recall, f1 = knn_classifier(X_train_c, y_train_c)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9489838756391104
Precision: 0.9339546526711997
Recall: 0.9665985902076585
F1 Score: 0.9499614076728744


In [173]:
X_train_corr, X_test_corr = correlation_method(X_train_c, X_test_c, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = knn_classifier(X_train_corr, y_train_c)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_c, y_train_c, X_test_c, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = knn_classifier(X_train_mi, y_train_c)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_c, X_test_c, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = knn_classifier(X_train_cov, y_train_c)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_c, X_test_c, y_train_c, y_test_c):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_c, X_test_c, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = knn_classifier(X_train_pca, y_train_c)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_c, X_test_c, y_train_c, y_test_c)



Accuracy with Correlation method: 0.9511958255789391
Precision with Correlation method: 0.9365470651107687
Recall with Correlation method: 0.968254905696323
F1 Score with Correlation method: 0.9521064834779338
Accuracy with Mutual Information method: 0.9417950622311053
Precision with Mutual Information method: 0.9293595191216436
Recall with Mutual Information method: 0.9566637454753286
F1 Score with Mutual Information method: 0.9427703246286285
Accuracy with Covariance method: 0.7501721977363383
Precision with Covariance method: 0.7172090206801646
Recall with Covariance method: 0.8283162507144219
F1 Score with Covariance method: 0.7686042717704863
Accuracy with PCA: 0.8501290885611572
Precision with PCA: 0.8319803238850965
Recall with PCA: 0.878274337969137
F1 Score with PCA: 0.8544698223551498


### LOGISTIC REGRESSION

In [174]:
accuracy, precision, recall, f1 = logistic_regression_classifier(X_train_c, y_train_c)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9119295800631058
Precision: 0.9088631756409153
Recall: 0.9160830634406553
F1 Score: 0.9123778886505887


In [175]:
X_train_corr, X_test_corr = correlation_method(X_train_c, X_test_c, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = logistic_regression_classifier(X_train_corr, y_train_c)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_c, y_train_c, X_test_c, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = logistic_regression_classifier(X_train_mi, y_train_c)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_c, X_test_c, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = logistic_regression_classifier(X_train_cov, y_train_c)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_c, X_test_c, y_train_c, y_test_c):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_c, X_test_c, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = logistic_regression_classifier(X_train_pca, y_train_c)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_c, X_test_c, y_train_c, y_test_c)



Accuracy with Correlation method: 0.9127592644102693
Precision with Correlation method: 0.9101469288755195
Recall with Correlation method: 0.9163593065345779
F1 Score with Correlation method: 0.9131583339856914
Accuracy with Mutual Information method: 0.9073677499400199
Precision with Mutual Information method: 0.9035477484621962
Recall with Mutual Information method: 0.9124964755191464
F1 Score with Mutual Information method: 0.9079235049399406
Accuracy with Covariance method: 0.7751963570357328
Precision with Covariance method: 0.731652827313823
Recall with Covariance method: 0.8711007811011621
F1 Score with Covariance method: 0.7951973161059521
Accuracy with PCA: 0.7761639716263247
Precision with PCA: 0.731529821284808
Recall with PCA: 0.8744137931034481
F1 Score with PCA: 0.7965095489393091


### RANDOM FOREST

In [182]:
accuracy, precision, recall, f1 = random_forest_classifier(X_train_c, y_train_c)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9818883156929823
Precision: 0.9794168644290275
Recall: 0.9845418174890455
F1 Score: 0.9819668645644359


In [183]:
X_train_corr, X_test_corr = correlation_method(X_train_c, X_test_c, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = random_forest_classifier(X_train_corr, y_train_c)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_c, y_train_c, X_test_c, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = random_forest_classifier(X_train_mi, y_train_c)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_c, X_test_c, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = random_forest_classifier(X_train_cov, y_train_c)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_c, X_test_c, y_train_c, y_test_c):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_c, X_test_c, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = random_forest_classifier(X_train_pca, y_train_c)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_c, X_test_c, y_train_c, y_test_c)



Accuracy with Correlation method: 0.9818880289357196
Precision with Correlation method: 0.9781051383542343
Recall with Correlation method: 0.9859203657839588
F1 Score with Correlation method: 0.9819871956935067
Accuracy with Mutual Information method: 0.9720727101715669
Precision with Mutual Information method: 0.972165065988141
Recall with Mutual Information method: 0.9721215469613259
F1 Score with Mutual Information method: 0.9721256936626365
Accuracy with Covariance method: 0.6807679550670487
Precision with Covariance method: 0.6585755783881877
Recall with Covariance method: 0.7537942465231474
F1 Score with Covariance method: 0.7028828333362236
Accuracy with PCA: 0.8540010762955932
Precision with PCA: 0.8413172831925116
Recall with PCA: 0.8733053915031433
F1 Score with PCA: 0.8569828466023488


### SVM

In [186]:
accuracy, precision, recall, f1 = svm_classifier(X_train_c, y_train_c)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.7486496122563878
Precision: 0.6794697111091198
Recall: 0.9434128405410555
F1 Score: 0.7899519703055607


In [193]:
X_train_corr, X_test_corr = correlation_method(X_train_c, X_test_c, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = svm_classifier(X_train_corr, y_train_c)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_c, y_train_c, X_test_c, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = svm_classifier(X_train_mi, y_train_c)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_c, X_test_c, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = svm_classifier(X_train_cov, y_train_c)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_c, X_test_c, y_train_c, y_test_c):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_c, X_test_c, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = svm_classifier(X_train_pca, y_train_c)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_c, X_test_c, y_train_c, y_test_c)



Accuracy with Correlation method: 0.7487878292570789
Precision with Correlation method: 0.6796771042696816
Recall with Correlation method: 0.9431369784720898
F1 Score with Correlation method: 0.7899949931648912
Accuracy with Mutual Information method: 0.7489260462577699
Precision with Mutual Information method: 0.6798157001956909
Recall with Mutual Information method: 0.9431369784720898
F1 Score with Mutual Information method: 0.7900876371134739
Accuracy with Covariance method: 0.783628645521186
Precision with Covariance method: 0.721543210897886
Recall with Covariance method: 0.9254734235092398
F1 Score with Covariance method: 0.810825401937237
Accuracy with PCA: 0.809898287198869
Precision with PCA: 0.7527546200254758
Recall with PCA: 0.9240929700895408
F1 Score with PCA: 0.8296410788046288


### NAIVE BAYES

In [189]:
accuracy, precision, recall, f1 = naive_bayes_classifier(X_train_s, y_train_s)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8940801830658366
Precision: 0.8820494317841793
Recall: 0.9119341563786009
F1 Score: 0.8966292389994871


In [191]:
X_train_corr, X_test_corr = correlation_method(X_train_c, X_test_c, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = naive_bayes_classifier(X_train_corr, y_train_c)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_c, y_train_c, X_test_c, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = naive_bayes_classifier(X_train_mi, y_train_c)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_c, X_test_c, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = naive_bayes_classifier(X_train_cov, y_train_c)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_c, X_test_c, y_train_c, y_test_c):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_c, X_test_c, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = naive_bayes_classifier(X_train_pca, y_train_c)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_c, X_test_c, y_train_c, y_test_c)



Accuracy with Correlation method: 0.9167699470741678
Precision with Correlation method: 0.9036429301022365
Recall with Correlation method: 0.9334745665841113
F1 Score with Correlation method: 0.9182545901887037
Accuracy with Mutual Information method: 0.9101338104974187
Precision with Mutual Information method: 0.8911898986520588
Recall with Mutual Information method: 0.934854257953896
F1 Score with Mutual Information method: 0.9124283077535327
Accuracy with Covariance method: 0.7745044117604889
Precision with Covariance method: 0.7084597591935988
Recall with Covariance method: 0.9345814440845875
F1 Score with Covariance method: 0.8059224506795641
Accuracy with PCA: 0.7803105772328115
Precision with PCA: 0.714667144128644
Recall with PCA: 0.9351331682225186
F1 Score with PCA: 0.8100854969682691


### DECISION TREE

In [190]:
accuracy, precision, recall, f1 = decision_tree_classifier(X_train_s, y_train_s)

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9589305292296457
Precision: 0.956865098183407
Recall: 0.9618655692729767
F1 Score: 0.9593537433856784


In [192]:
X_train_corr, X_test_corr = correlation_method(X_train_c, X_test_c, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = decision_tree_classifier(X_train_corr, y_train_c)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_c, y_train_c, X_test_c, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = decision_tree_classifier(X_train_mi, y_train_c)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_c, X_test_c, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = decision_tree_classifier(X_train_cov, y_train_c)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_c, X_test_c, y_train_c, y_test_c):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_c, X_test_c, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = decision_tree_classifier(X_train_pca, y_train_c)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_c, X_test_c, y_train_c, y_test_c)



Accuracy with Correlation method: 0.9712431214101576
Precision with Correlation method: 0.9692592945621286
Recall with Correlation method: 0.9735008573061535
F1 Score with Correlation method: 0.9713621913426539
Accuracy with Mutual Information method: 0.9713817207538658
Precision with Mutual Information method: 0.9690270722847414
Recall with Mutual Information method: 0.9740529624690417
F1 Score with Mutual Information method: 0.9715160091393129
Accuracy with Covariance method: 0.6274021416944104
Precision with Covariance method: 0.626399086264661
Recall with Covariance method: 0.6356639359878072
F1 Score with Covariance method: 0.6308122254204772
Accuracy with PCA: 0.8031227865923775
Precision with PCA: 0.7990847421530036
Recall with PCA: 0.8109236044960945
F1 Score with PCA: 0.8048874525652334


## 3- BALANCING TECHNIQUE: ENSEMBLE METHODS

### SAMPLING

In [195]:
from imblearn.ensemble import BalancedRandomForestClassifier
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
import pandas as pd

# Define your ensemble classifier, for example, Balanced Random Forest
ensemble_clf = BalancedRandomForestClassifier(n_estimators=100, random_state=42)

# Fit the ensemble classifier to your original dataset
ensemble_clf.fit(X, y)

# Use RandomOverSampler to generate synthetic samples to balance the dataset
ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X, y)

# Split the resampled data into training and testing sets
X_train_e, X_test_e, y_train_e, y_test_e = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Count the occurrences of each class label in the resampled training and testing sets
train_balance = pd.Series(y_train_s).value_counts()
test_balance = pd.Series(y_test_s).value_counts()

# Print the class distribution
print("Class distribution in the resampled training set:")
print(train_balance)

print("\nClass distribution in the resampled testing set:")
print(test_balance)

# Create a DataFrame for the resampled data
resampled_data = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)

# Define the column names
column_names = list(X.columns) + ['Personal Loan']

# Set the column names
resampled_data.columns = column_names

# Save the resampled data to a CSV file
resampled_data.to_excel('Bank_Loan_ENSEMBLE.xlsx', index=False)

C:\Users\DELL\anaconda3\Lib\site-packages\imblearn\ensemble\_forest.py:546: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(
C:\Users\DELL\anaconda3\Lib\site-packages\imblearn\ensemble\_forest.py:558: FutureWarning: The default of `replacement` will change from `False` to `True` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `True` to silence this warning and adopt the future behaviour.
  warn(


Class distribution in the resampled training set:
Personal Loan
1    3645
0    3587
Name: count, dtype: int64

Class distribution in the resampled testing set:
Personal Loan
0    933
1    875
Name: count, dtype: int64


### KNN

In [196]:
accuracy, precision, recall, f1 = knn_classifier(X_train_e, y_train_e)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9590700844308968
Precision: 0.9332602132748973
Recall: 0.9895747599451303
F1 Score: 0.9605894607539118


In [198]:
X_train_corr, X_test_corr = correlation_method(X_train_e, X_test_e, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = knn_classifier(X_train_corr, y_train_e)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_e, y_train_e, X_test_e, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = knn_classifier(X_train_mi, y_train_e)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_e, X_test_e, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = knn_classifier(X_train_cov, y_train_e)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_e, X_test_e, y_train_e, y_test_e):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_e, X_test_e, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = knn_classifier(X_train_pca, y_train_e)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_e, X_test_e, y_train_e, y_test_e)

Accuracy with Correlation method: 0.9616977368160959
Precision with Correlation method: 0.9376292798024293
Recall with Correlation method: 0.989849108367627
F1 Score with Correlation method: 0.963030583426745
Accuracy with Mutual Information method: 0.9662605227967245
Precision with Mutual Information method: 0.9443815513824898
Recall with Mutual Information method: 0.9914951989026063
F1 Score with Mutual Information method: 0.967353542215848
Accuracy with Covariance method: 0.7975663866959923
Precision with Covariance method: 0.7289458805559323
Recall with Covariance method: 0.9528120713305898
F1 Score with Covariance method: 0.8259604321169913
Accuracy with PCA: 0.8072446354885052
Precision with PCA: 0.7397100915451262
Recall with PCA: 0.9528120713305899
F1 Score with PCA: 0.832841464491292


### LOGISTIC REGRESSION

In [201]:
accuracy, precision, recall, f1 = logistic_regression_classifier(X_train_e, y_train_e)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.8926996380167486
Precision: 0.8929897488406386
Recall: 0.8943758573388203
F1 Score: 0.8936365831243723


In [202]:
X_train_corr, X_test_corr = correlation_method(X_train_e, X_test_e, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = logistic_regression_classifier(X_train_corr, y_train_e)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_e, y_train_e, X_test_e, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = logistic_regression_classifier(X_train_mi, y_train_e)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_e, X_test_e, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = logistic_regression_classifier(X_train_cov, y_train_e)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_e, X_test_e, y_train_e, y_test_e):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_e, X_test_e, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = logistic_regression_classifier(X_train_pca, y_train_e)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_e, X_test_e, y_train_e, y_test_e)

Accuracy with Correlation method: 0.8935292267781578
Precision with Correlation method: 0.8931297467139261
Recall with Correlation method: 0.8960219478737997
F1 Score with Correlation method: 0.8945497585041775
Accuracy with Mutual Information method: 0.894911492370823
Precision with Mutual Information method: 0.8949180380282012
Recall with Mutual Information method: 0.8968449931412895
F1 Score with Mutual Information method: 0.8958523820429276
Accuracy with Covariance method: 0.5178379267067552
Precision with Covariance method: 0.5190102390613788
Recall with Covariance method: 0.5909465020576131
F1 Score with Covariance method: 0.5525715154395836
Accuracy with PCA: 0.5221245654432646
Precision with PCA: 0.5227814765617547
Recall with PCA: 0.597530864197531
F1 Score with PCA: 0.5576088174589386


### RANDOM FOREST

In [203]:
accuracy, precision, recall, f1 = random_forest_classifier(X_train_e, y_train_e)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9950219895027725
Precision: 0.9902254518947025
Recall: 1.0
F1 Score: 0.9950873664658804


In [204]:
X_train_corr, X_test_corr = correlation_method(X_train_e, X_test_e, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = random_forest_classifier(X_train_corr, y_train_e)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_e, y_train_e, X_test_e, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = random_forest_classifier(X_train_mi, y_train_e)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_e, X_test_e, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = random_forest_classifier(X_train_cov, y_train_e)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_e, X_test_e, y_train_e, y_test_e):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_e, X_test_e, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = random_forest_classifier(X_train_pca, y_train_e)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_e, X_test_e, y_train_e, y_test_e)

Accuracy with Correlation method: 0.9954367360906
Precision with Correlation method: 0.9910310977541
Recall with Correlation method: 1.0
F1 Score with Correlation method: 0.9954945034740404
Accuracy with Mutual Information method: 0.9952983279184003
Precision with Mutual Information method: 0.99076814180736
Recall with Mutual Information method: 1.0
F1 Score with Mutual Information method: 0.9953602112891315
Accuracy with Covariance method: 0.9300328528237467
Precision with Covariance method: 0.8781622317691831
Recall with Covariance method: 1.0
F1 Score with Covariance method: 0.9351131736923298
Accuracy with PCA: 0.94980677339772
Precision with PCA: 0.9094416930175114
Recall with PCA: 1.0
F1 Score with PCA: 0.9525706494725628


### SVM

In [209]:
accuracy, precision, recall, f1 = svm_classifier(X_train_e, y_train_e)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.5040100135636185
Precision: 0.5040100135636185
Recall: 1.0
F1 Score: 0.6702216024340769


In [210]:
X_train_corr, X_test_corr = correlation_method(X_train_e, X_test_e, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = svm_classifier(X_train_corr, y_train_e)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_e, y_train_e, X_test_e, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = svm_classifier(X_train_mi, y_train_e)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_e, X_test_e, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = svm_classifier(X_train_cov, y_train_e)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_e, X_test_e, y_train_e, y_test_e):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_e, X_test_e, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = svm_classifier(X_train_pca, y_train_e)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_e, X_test_e, y_train_e, y_test_e)

Accuracy with Correlation method: 0.5040100135636185
Precision with Correlation method: 0.5040100135636185
Recall with Correlation method: 1.0
F1 Score with Correlation method: 0.6702216024340769
Accuracy with Mutual Information method: 0.5040100135636185
Precision with Mutual Information method: 0.5040100135636185
Recall with Mutual Information method: 1.0
F1 Score with Mutual Information method: 0.6702216024340769
Accuracy with Covariance method: 0.5145210054474321
Precision with Covariance method: 0.5143225584566682
Recall with Covariance method: 0.6685871056241427
F1 Score with Covariance method: 0.5768928946039649
Accuracy with PCA: 0.5315272405061838
Precision with PCA: 0.5290571383362573
Recall with PCA: 0.6419753086419753
F1 Score with PCA: 0.5787363454747584


### NAIVE BAYES

In [205]:
accuracy, precision, recall, f1 = naive_bayes_classifier(X_train_e, y_train_e)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.826741261789308
Precision: 0.8664810177038346
Recall: 0.7758573388203017
F1 Score: 0.8186057641244516


In [206]:
X_train_corr, X_test_corr = correlation_method(X_train_e, X_test_e, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = naive_bayes_classifier(X_train_corr, y_train_e)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_e, y_train_e, X_test_e, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = naive_bayes_classifier(X_train_mi, y_train_e)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_e, X_test_e, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = naive_bayes_classifier(X_train_cov, y_train_e)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_e, X_test_e, y_train_e, y_test_e):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_e, X_test_e, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = naive_bayes_classifier(X_train_pca, y_train_e)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_e, X_test_e, y_train_e, y_test_e)

Accuracy with Correlation method: 0.8279858838958076
Precision with Correlation method: 0.8670698924076923
Recall with Correlation method: 0.7780521262002743
F1 Score with Correlation method: 0.8201012603170056
Accuracy with Mutual Information method: 0.8318575848729809
Precision with Mutual Information method: 0.8673098920731666
Recall with Mutual Information method: 0.7868312757201646
F1 Score with Mutual Information method: 0.8250616320187262
Accuracy with Covariance method: 0.5161795138699709
Precision with Covariance method: 0.5152097050156053
Recall with Covariance method: 0.6784636488340192
F1 Score with Covariance method: 0.5855443803241709
Accuracy with PCA: 0.5171486578326313
Precision with PCA: 0.5150497367501381
Recall with PCA: 0.7286694101508916
F1 Score with PCA: 0.6032681637458415


### DESICION TREE

In [207]:
accuracy, precision, recall, f1 = decision_tree_classifier(X_train_e, y_train_e)
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)

Accuracy: 0.9922565980456537
Precision: 0.9848785966647895
Recall: 1.0
F1 Score: 0.9923792688779544


In [208]:
X_train_corr, X_test_corr = correlation_method(X_train_e, X_test_e, threshold=0.8)
accuracy_corr, precision_corr, recall_corr, f1_corr = decision_tree_classifier(X_train_corr, y_train_e)
print("Accuracy with Correlation method:", accuracy_corr)
print("Precision with Correlation method:", precision_corr)
print("Recall with Correlation method:", recall_corr)
print("F1 Score with Correlation method:", f1_corr)

X_train_mi, X_test_mi = mutual_information(X_train_e, y_train_e, X_test_e, n_features=10)
accuracy_mi, precision_mi, recall_mi, f1_mi = decision_tree_classifier(X_train_mi, y_train_e)
print("Accuracy with Mutual Information method:", accuracy_mi)
print("Precision with Mutual Information method:", precision_mi)
print("Recall with Mutual Information method:", recall_mi)
print("F1 Score with Mutual Information method:", f1_mi)

X_train_cov, X_test_cov = covariance_method(X_train_e, X_test_e, threshold=0.8)
accuracy_cov, precision_cov, recall_cov, f1_cov = decision_tree_classifier(X_train_cov, y_train_e)
print("Accuracy with Covariance method:", accuracy_cov)
print("Precision with Covariance method:", precision_cov)
print("Recall with Covariance method:", recall_cov)
print("F1 Score with Covariance method:", f1_cov)

def main(X_train_e, X_test_e, y_train_e, y_test_e):
    # Perform PCA
    n_components = 2  # Number of components to retain
    X_train_pca, X_test_pca = perform_pca(X_train_e, X_test_e, n_components)
    
    # Evaluate Logistic classifier using PCA
    accuracy, precision, recall, f1 = decision_tree_classifier(X_train_pca, y_train_e)
    
    print("Accuracy with PCA:", accuracy)
    print("Precision with PCA:", precision)
    print("Recall with PCA:", recall)
    print("F1 Score with PCA:", f1)

if __name__ == "__main__":
    main(X_train_e, X_test_e, y_train_e, y_test_e)

Accuracy with Correlation method: 0.9919800684585173
Precision with Correlation method: 0.9843446305778552
Recall with Correlation method: 1.0
F1 Score with Correlation method: 0.9921086430618979
Accuracy with Mutual Information method: 0.9834065998139901
Precision with Mutual Information method: 0.968169285285256
Recall with Mutual Information method: 1.0
F1 Score with Mutual Information method: 0.9838166152151503
Accuracy with Covariance method: 0.9300328528237466
Precision with Covariance method: 0.878162231769183
Recall with Covariance method: 1.0
F1 Score with Covariance method: 0.9351131736923298
Accuracy with PCA: 0.9319696113769989
Precision with PCA: 0.8811612080672873
Recall with PCA: 1.0
F1 Score with PCA: 0.9368035890489343
